## MK_Skel Dataset Analysis
- Wells: A6 - DMSO, C6 - Taxol
- Scanned over 22 hours * 3 stacks = 66 images total
- Analyzing hour 4

In [1]:
import os
import sys
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
# Search working dir for folders/files
for path, dirs, files in os.walk(os.getcwd()):
    for file in files:
          print("Found %s" % os.path.join(path, file))

Found C:\Users\Prakrith\Documents\Github\jupyter\italiano-lab\mk-skel\skel-analysis.ipynb
Found C:\Users\Prakrith\Documents\Github\jupyter\italiano-lab\mk-skel\.ipynb_checkpoints\skel-analysis-checkpoint.ipynb
Found C:\Users\Prakrith\Documents\Github\jupyter\italiano-lab\mk-skel\.ipynb_checkpoints\Skel_Analysis-checkpoint.ipynb
Found C:\Users\Prakrith\Documents\Github\jupyter\italiano-lab\mk-skel\results\EdgesResults.csv
Found C:\Users\Prakrith\Documents\Github\jupyter\italiano-lab\mk-skel\results\results_proplatelets.csv
Found C:\Users\Prakrith\Documents\Github\jupyter\italiano-lab\mk-skel\results\results_prot_seed.csv
Found C:\Users\Prakrith\Documents\Github\jupyter\italiano-lab\mk-skel\results\VerticesResults.csv
Found C:\Users\Prakrith\Documents\Github\jupyter\italiano-lab\mk-skel\results\branchpoints\180417_ARW_A6_1-0003_branchpoints.png
Found C:\Users\Prakrith\Documents\Github\jupyter\italiano-lab\mk-skel\results\branchpoints\180417_ARW_C6_1-0018_branchpoints.png
Found C:\Users\P

In [3]:
def parse_skel(edge, vert, imnum):
    # filter for Image 4
    edge = edge[edge['ImageNumber'] == imnum]
    vert = vert[vert['ImageNumber'] == imnum]
    
    # concat x & y coordinates into one column
    vert['xy'] = vert['x'].astype(str) + ', ' + vert['y'].astype(str)

    # create dicts, Node is key, vert columns are values
    xydict = dict(zip(vert.Node, vert.xy))
    labeldict = dict(zip(vert.Node, vert.Labels))
    kinddict = dict(zip(vert.Node, vert.Kind))
    
    # use xy coord dict, and add values to edge for Node_1 & Node_2
    edge['Node_1_xy'] = edge['Node_1'].map(xydict)
    edge['Node_2_xy'] = edge['Node_2'].map(xydict)
    edge['Node_1_Label'] = edge['Node_1'].map(labeldict)
    edge['Node_2_Label'] = edge['Node_2'].map(labeldict)
    edge['Node_1_Kind'] = edge['Node_1'].map(kinddict)
    edge['Node_2_Kind'] = edge['Node_2'].map(kinddict)
    
    edge.to_csv('UpdatedEdges.csv')

In [4]:
# Filter by Well_Site, and place in dfs
def split_wells(df):
    d = []
    wells = df.Metadata_Well_Site.unique()
    for well in wells:
        well_df = df[df['Metadata_Well_Site'].str.contains(well)]
        d.append(well_df)
    return d

### Branch Lengths
- Aggregate Average & Total lengths for DMSO/Taxol

In [5]:
epath = os.getcwd() + os.sep + "results" + os.sep + "EdgesResults.csv"
vpath = os.getcwd() + os.sep + "results" + os.sep + "VerticesResults.csv"

In [6]:
edge = pd.read_csv(epath)
vert = pd.read_csv(vpath)

In [7]:
# DMSO
parse_skel(edge, vert, 4)

c:\users\prakrith\.virtualenvs\jupyter-tcjg40cp\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [8]:
# Taxol
parse_skel(edge, vert, 41)

### Area Results

In [9]:
path = os.getcwd() + os.sep + "results" + os.sep + "results_proplatelets.csv"

In [10]:
df = pd.read_csv(path)
# Remove AreaShape_ from column headers
df.columns = df.columns.str.replace('AreaShape_', '')

In [11]:
dmso = df[df["ImageNumber"] == 4]
taxol = df[df["ImageNumber"] == 107]

In [12]:
dmso.to_csv('dmso-data.csv')
taxol.to_csv('taxol-data.csv')